# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# План проекта: #

1. [Подготовка данных](#cell1)
2. [Предобработка данных](#cell2)
3. [Ответы на вопросы](#cell3)
4. [Общий вывод](#cell4)

## Подготовка данных
<a id='cell1'></a>

**Импортируем библиотеки:**

In [1]:
import pandas as pd

**Прочитаем приложенный файл и сохраним в переменную:**

In [2]:
df = pd.read_csv('/datasets/data.csv')

**Первые строки таблицы:**

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Общая информация о данных:**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, с различными типами данных у каждого столбца - float64(2 столбца), int64(5 столбцов), object(5 столбцов).

Подробно разберём, какие в df столбцы и какую информацию они содержат:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть пустые значения.

**Вывод**

Каждая строка таблицы содержит информацию о клиенте банка и информацию о нём. Две проблемы, которые нужно решать: пропуски и некорректные данные. Для проверки рабочих гипотез особенно ценны столбцы family_status, children, dob_years, education и total_income. 

## Предобработка данных
<a id='cell2'></a>

### Обработка пропусков

Приведем в единый формат столбец образование.

In [5]:
df['education'] = df['education'].str.lower()

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [6]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Посчитаем медиану столбца total_income и медиану столбца days_employed, и заменим пустые значения - медианами по каждому столбцу.

In [7]:
df['total_income'] = df.groupby(['income_type', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))

In [8]:
df['days_employed'] = df.groupby(['income_type', 'education'])['days_employed'].apply(lambda x: x.fillna(x.median()))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Исправим некорректные значения в столбце 'children', вместо значения '-1' укажем значение '0', вместо значения '20' укажем значение '2'.

In [10]:
df.loc[df['children'] == -1, 'children'] = 0
df.loc[df['children'] == 20, 'children'] = 2

In [11]:
print(df['children'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Исправим некорректные значения в столбце 'dob_years', вместо значения '0' укажем среднее значение столбца.

In [12]:
dob_years_avg = df['dob_years'].mean()
df.loc[df['dob_years'] == 0, 'dob_years'] = dob_years_avg

Исправим некорректные значения в столбце 'days_employed', заменим отрицательные числа на положительные в стаже.

In [30]:
df['days_employed'] = df['days_employed'].abs()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,dob_years_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,группа 31-45,выше_среднего доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,группа 31-45,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,группа 31-45,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,группа 31-45,выше_среднего доход
4,0,1989,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,группа 46-60,средний доход


Наблюдаются некорректные знаения, которые превышают стаж 50 лет(18250), все значения, которые превышают указанный стаж, заменим на медианное значение по столбцу.

In [14]:
days_employed_median = df['days_employed'].median()

df.loc[df['days_employed'] >= 18250, 'days_employed'] = days_employed_median
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,1989.992107,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Обработка дубликатов

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [15]:
df.duplicated().sum()

71

In [16]:
df = df.drop_duplicates()

In [17]:
df.duplicated().sum()

0

### Замена типа данных

Исправим типы данных в столбцах: 'days_employed', 'total_income', 'dob_years', присвоим тип данных - 'int'.

In [18]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Лемматизация

In [19]:

from pymystem3 import Mystem
m = Mystem()
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

purpose_list = df['purpose'].unique()

lemmas = []

for i in purpose_list:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)


def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'автомоб' in i:
            return 'автомобиль'
        elif 'жил' in i:
            return  'недвижимость'
        elif 'недвиж' in i:
            return  'недвижимость'
        elif 'свад' in i:
            return  'свадьба'
        elif 'образов' in i:
            return 'образование'

df['purpose_def'] = df['purpose'].apply(purpose_change)        
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,1989,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

После лемматизации, в таблице добавился новый столбец 'purpose_def', с помощью которого мы опредилили четыре основных цели получения кредита, с помощью которого гораздо легче понять: Есть ли зависимость между целью кредита и возвратом кредита в срок?

### Категоризация данных

В данном шаге мы разделим клиентов на категории по возрасту, чтобы в дальнейшем можно было ответить на вопрос: Есть ли зависимость между возрастом и возвратом кредита в срок

In [20]:
def years_group(year):
    if year <= 30:
        return 'группа 18-30'
    elif year <= 45:
        return 'группа 31-45'
    elif year <= 60:
        return 'группа 46-60'
    else:
        return 'группа за 60'


df['dob_years_group'] = df['dob_years'].apply(years_group)
df['dob_years_group'].value_counts()

группа 31-45    8587
группа 46-60    7024
группа 18-30    3717
группа за 60    2126
Name: dob_years_group, dtype: int64

Создадим отдельный «словарь», где названию категории будет соответствовать номер. И в будущих таблицах будем обращаться уже не к длинной строке, а к её числовому обозначению.

In [21]:
dict_education = df[['education', 'education_id']]

dict_education = dict_education.drop_duplicates().reset_index(drop=True)
dict_education.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [22]:
dict_family = df[['family_status', 'family_status_id']]

dict_family = dict_education.drop_duplicates().reset_index(drop=True)
dict_family.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


## Ответы на вопросы
<a id='cell3'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
df.groupby('children').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
          count   sum <lambda_0>
children                        
0         14138  1064      7.53%
1          4808   444      9.23%
2          2128   202      9.49%
3           330    27      8.18%
4            41     4      9.76%
5             9     0       0.0%

**Вывод**

Клиенты, у которых нет детей платят лучше, чем те, у кого дети есть. 

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
df.groupby('purpose_def').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
              count  sum <lambda_0>
purpose_def                        
автомобиль     4306  403      9.36%
недвижимость  10811  782      7.23%
образование    4013  370      9.22%
свадьба        2324  186       8.0%

**Вывод**

Клиенты, у которых цель кредита связана с 'недвижимостью' платят лучше остальных. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
df.groupby('family_status').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                       count  sum <lambda_0>
family_status                               
Не женат / не замужем   2810  274      9.75%
в разводе               1195   85      7.11%
вдовец / вдова           959   63      6.57%
гражданский брак        4151  388      9.35%
женат / замужем        12339  931      7.55%

**Вывод**

Клиенты, со статусом 'Не женат / не замужем', 'гражданский брак' относятся к возврату кредита хуже, чем все остальные. Те, кто вступал в официальный брак, платят значительно лучше.

- Есть ли зависимость между типом занятости и возвратом кредита в срок?

In [26]:
df.groupby('income_type').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
                 count   sum <lambda_0>
income_type                            
безработный          2     1      50.0%
в декрете            1     1     100.0%
госслужащий       1457    86       5.9%
компаньон         5078   376       7.4%
пенсионер         3829   216      5.64%
предприниматель      2     0       0.0%
сотрудник        11084  1061      9.57%
студент              1     0       0.0%

**Вывод**

Клиенты, со статусом 'пенсионер', 'госслужащий' относятся к возврату кредита более ответственно, а значит и платят значительно лучше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
def income_group(value):
    if value < 100000:
        return 'минимальный доход'
    if value <= 250000:
        return 'средний доход'
    if value <= 500000:
        return 'выше_среднего доход'
    if value > 500000:
        return 'максимальный доход'
    
df['total_income_group'] = df['total_income'].apply(income_group)       

df.groupby('total_income_group').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
                     count   sum <lambda_0>
total_income_group                         
выше_среднего доход   2591   180      6.95%
максимальный доход     222    14      6.31%
минимальный доход     4463   354      7.93%
средний доход        14178  1193      8.41%

**Вывод**

Клиенты с доходом выше среднего возвращают кредит в срок немного лучше.

- Есть ли зависимость между уровнем образования и возвратом кредита в срок?

In [28]:
df.groupby('education').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
                     count   sum <lambda_0>
education                                  
высшее                5250   278       5.3%
начальное              282    31     10.99%
неоконченное высшее    744    68      9.14%
среднее              15172  1364      8.99%
ученая степень           6     0       0.0%

**Вывод**

Присутствует прямая зависимости между уровнем образования и возвратом кредита в срок. Клиенты, которые имеют высшее образование, возвращают кредит в срок значительно лучше.

- Есть ли зависимость между возрастом и возвратом кредита в срок?

In [29]:
df.groupby('dob_years_group').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                count  sum <lambda_0>
dob_years_group                      
группа 18-30     3717  403     10.84%
группа 31-45     8587  770      8.97%
группа 46-60     7024  467      6.65%
группа за 60     2126  101      4.75%

**Вывод**

Присутствует прямая зависимости между возрастом и возвратом кредита в срок. Чем старше клиенты, тем лучше они относятся к исполнению своих обязательств.

## Общий вывод
<a id='cell4'></a>

- Общий вывод: семейный статус влияет на вероятность платежей по кредиту в срок - клиенты, находящиеся в браке чаще платят в срок - (имели задолженность: 7.55% ) чем те, кто в браке не состоит - (имели задолженность: 9.75% ), или находятся в гражданском браке - (имели задолженность: 9.35%). 
- Клиенты, не имеющие детей производят оплату более своевременно - (имели задолженность: 7.53% ), чем клиенты с детьми - (имели задолженность: более 8.18%). 
- Найдена прямая зависимость между уровнем образования и возвратом кредита в срок. Клиенты, которые имеют 'высшее' образование возвращают кредит в срок значительно лучше - (имели задолженность: 5.3% ), чем клиенты, у которых образование 'среднее' - (имели задолженность: 8.99% ). Также найдена прямая зависимость между возрастом и возвратом кредита в срок. 
- Чем старше клиент, тем более ответственно относится к своим обязательствам. Клиенты в возрастной группе 'от 18 до 30' - (имели задолженность: 10.84% ), в возрастной группе 'от 31 до 45' - (имели задолженность: 8.97% ), в возрастной группе 'от 46 до 60' - (имели задолженность: 6.65% ), в возрастной группе 'от 61 до 100' - (имели задолженность: 4.75% ).